In [ ]:
import os
import os.path as osp
import sys
import torch
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import random
import math
import pickle
import json
import time
import re
import pickle
from Bio import SeqIO
from tqdm import tqdm
%matplotlib inline
import sklearn.metrics as metrics
from torch_geometric.nn import MessagePassing, SAGEConv
from torch import nn
from scipy.stats import norm
from networkx.algorithms import bipartite
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

/home/hrren/miniconda3/envs/pyg/lib/python3.9/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/hrren/miniconda3/envs/pyg/lib/python3.9/site-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(


In [5]:
df = pd.read_csv('goldstand775pairs.csv')
df

,TF,Target,TF_pep,TF_cds,Target_promoter,Edge_attr0,Edge_attr2,Edge_attr3,Edge_attr4,Edge_attr5,...,Edge_attr28,Edge_attr29,Edge_attr30,Edge_attr31,Edge_attr32,PWM,cPWM,GENIE3,ATAC,Chip
0,LOC_Os03g60430,LOC_Os06g06760,MLLDLNVESPERSGTSSSSVLNSGDAGGGGGGGGGGGLFRFDLLAS...,ATGTTGTTGGATCTCAATGTGGAGTCGCCGGAACGGTCCGGCACGT...,GTGGCATCGCACTACACACACCACCCGTGAGGACTGAGGAGTCTCG...,0.684466,0.930658,0.031236,0.985271,0.683051,...,0.769121,0.816782,0.690776,0.590202,0.350191,0.628816,0.695014,0.007306,0.761364,0.000000
1,LOC_Os03g60430,LOC_Os12g44360,MLLDLNVESPERSGTSSSSVLNSGDAGGGGGGGGGGGLFRFDLLAS...,ATGTTGTTGGATCTCAATGTGGAGTCGCCGGAACGGTCCGGCACGT...,TCAAAACTTCCAAACGACACGTGTTTTTTTAAAAAATCTACCTAGA...,0.743212,0.857745,0.999184,0.950132,0.682092,...,0.954925,0.605103,0.024517,0.999660,0.834553,0.695014,0.683667,0.005444,0.761364,0.000000
2,LOC_Os06g43220,LOC_Os01g64000,MCRDCGKQVYLGGFDTAQAAARAYDQAAIKFRGVEADINFTLDDYK...,ATGTGCAGGGATTGTGGGAAGCAGGTGTACTTGGGTGGATTCGATA...,GTTAGCTGGCACATCACCTATTATCTGATTGGGAGTGATTAAGATG...,0.000000,0.039289,0.780055,0.998819,0.746520,...,0.941253,0.114680,0.281077,0.825994,0.445571,0.683667,0.683667,0.011664,0.727271,0.772325
3,LOC_Os07g13170,LOC_Os01g62920,MVLDLNVESPGGSAATSSSSTPPPPPDGGGGGYFRFDLLGGSPDED...,ATGGTGCTGGATCTCAATGTGGAGTCGCCGGGTGGGTCGGCGGCGA...,AAATATGCCGGTTCTATATTTTTTTCATCGTGGGAAGGTGAGAAAA...,0.000000,0.219498,0.539522,0.598057,0.107919,...,0.688917,0.096707,0.199466,0.402903,0.558947,0.683667,0.696472,0.006627,0.785802,0.820221
4,LOC_Os07g13170,LOC_Os05g38120,MVLDLNVESPGGSAATSSSSTPPPPPDGGGGGYFRFDLLGGSPDED...,ATGGTGCTGGATCTCAATGTGGAGTCGCCGGGTGGGTCGGCGGCGA...,GTTTGGTTGAAATTGGAACGATGTGATGGAAAAGTTGAAAGTTTGT...,0.000000,0.392471,0.441903,0.743827,0.867580,...,0.824625,0.227214,0.260454,0.083318,0.583988,0.696472,0.000000,0.005194,0.785802,0.825706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,LOC_Os05g34310,LOC_Os08g25734,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,TATTCGTTGCTCGGAGATTGAGAGACAGACAGAGAGAGCATATTAG...,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
771,LOC_Os05g34310,LOC_Os06g06560,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,AAATCTAGCTAGAACAATCACGTTAACCCAGCTGAAACAAGCTGGC...,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
772,LOC_Os05g34310,LOC_Os08g09230,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,AAGATATGGCTTCATGAGAGAAGAAAGAATTAGCATTAACTACACT...,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
773,LOC_Os05g34310,LOC_Os05g45720,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,CCTTACATCAGTTGATGCTCTAGGAGTTGACACGTGGGATTCCAAC...,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
value_columns = df.columns[6:]
zero_rows = df[(df[value_columns] == 0).all(axis=1)]

In [7]:
zero_rows

,TF,Target,TF_pep,TF_cds,Target_promoter,Edge_attr0,Edge_attr2,Edge_attr3,Edge_attr4,Edge_attr5,...,Edge_attr28,Edge_attr29,Edge_attr30,Edge_attr31,Edge_attr32,PWM,cPWM,GENIE3,ATAC,Chip
569,LOC_Os02g57650,LOC_Os01g72150,MSQSPPDSSSAAAVPLAPGFRFHPTDEELVSYYLRRRILGRRLRID...,ATGAGCCAATCGCCGCCGGACTCCTCCTCCGCGGCGGCGGTGCCCC...,CCTTGGAGAAGAAGCAACACACATTTCAAATTTAAAATCCGAGTCA...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
570,LOC_Os03g17570,LOC_Os10g32600,MGSACEAGTDEPSRDDVKGTGNGILENGHSHKPEEEEWRNGMGEDL...,ATGGGTAGCGCCTGCGAAGCTGGTACGGACGAGCCTTCCCGAGACG...,GTCAAATTAGACTGCAAGTAGTAATTAAATCGAAGTCTTGGAGGGT...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,LOC_Os03g17570,LOC_Os06g51260,MGSACEAGTDEPSRDDVKGTGNGILENGHSHKPEEEEWRNGMGEDL...,ATGGGTAGCGCCTGCGAAGCTGGTACGGACGAGCCTTCCCGAGACG...,AGGAAAATCTTTAGGGTGGTTTCTAGAGGGGACGCGTGGTTTGCTT...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,LOC_Os01g18290,LOC_Os05g48300,MSDGNDFAELLWENGQAVVHGRKKHPQPAFPPFGFFGGTGGGGGGS...,AGGTCCCCCCACGCCACAGCCTCTTATCCCACACGCGGACCAACAG...,"""CTACACACTCCTCCTCCTCCTCCTCGCCGATAACGATGGCAATCG...",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
573,LOC_Os09g13940,LOC_Os06g03990,MSPALPPFLHGSELQSPPTVSSCSLLILLLQNWYTRGARTQQNPFT...,ATGTCGCCTGCTCTACCCCCGTTTCTCCATGGTTCTGAGCTCCAGT...,ATTCTCTCAGGGGAATCTCACAAGATCAAGAGAGTTTGCCCGATGC...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,LOC_Os05g34310,LOC_Os08g25734,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,TATTCGTTGCTCGGAGATTGAGAGACAGACAGAGAGAGCATATTAG...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
771,LOC_Os05g34310,LOC_Os06g06560,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,AAATCTAGCTAGAACAATCACGTTAACCCAGCTGAAACAAGCTGGC...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
772,LOC_Os05g34310,LOC_Os08g09230,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,AAGATATGGCTTCATGAGAGAAGAAAGAATTAGCATTAACTACACT...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
773,LOC_Os05g34310,LOC_Os05g45720,MAMQLSLPVLPTGFRFHPTDEELVINYLQRRATGLSCPIPIIADVE...,ATGGCAATGCAGCTGTCCTTGCCTGTCTTGCCAACGGGTTTTCGTT...,CCTTACATCAGTTGATGCTCTAGGAGTTGACACGTGGGATTCCAAC...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
